### CX Calibration with HPO under the new code architecture / workflow (DEC 2023)

In [1]:
import sys
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
import time
import yaml
import pickle
import optuna
module_path = os.path.abspath(os.path.join('/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control'))
if module_path not in sys.path:
    sys.path.append(module_path)

from quantumenvironment import QuantumEnvironment
from agent import Agent
from gate_level_abstraction import gate_q_env_config
from helper_functions import load_agent_from_yaml_file, create_agent_config
from ppo import make_train_ppo
from qconfig import QEnvConfig

import logging
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s INFO %(message)s", # hardcoded INFO level
    datefmt="%Y-%m-%d %H:%M:%S",
    stream=sys.stdout,
)

/Users/lukasvoss/Library/Python/3.9/lib/python/site-packages/qiskit_dynamics/dispatch/backends/jax.py:34: UserWarning: The functionality in the perturbation module of Qiskit Dynamics requires a JAX version <= 0.4.6, due to a bug in JAX versions > 0.4.6. For versions 0.4.4, 0.4.5, and 0.4.6, using the perturbation module functionality requires setting os.environ['JAX_JIT_PJIT_API_MERGE'] = '0' before importing JAX or Dynamics.
  warnings.warn(


Starting Rabi experiment for qubit 0...
Rabi experiment for qubit 0 done.
Starting Drag experiment for qubit 0...
Drag experiments done for qubit 0 done.
Starting Rabi experiment for qubit 1...
Rabi experiment for qubit 1 done.
Starting Drag experiment for qubit 1...


/Users/lukasvoss/Library/Python/3.9/lib/python/site-packages/lmfit/minimizer.py:810: RuntimeWarning: invalid value encountered in sqrt
  par.stderr = np.sqrt(self.result.covar[ivar, ivar])
/Users/lukasvoss/Library/Python/3.9/lib/python/site-packages/lmfit/minimizer.py:817: RuntimeWarning: invalid value encountered in sqrt
  (par.stderr * np.sqrt(self.result.covar[jvar, jvar])))


Drag experiments done for qubit 1 done.
All single qubit calibrations are done
Updated Instruction Schedule Map <InstructionScheduleMap(1Q instructions:
  q0: {'delay', 'tdg', 'reset', 'measure', 'z', 's', 't', 'rz', 'sdg', 'h', 'x', 'id', 'sx'}
  q1: {'delay', 'tdg', 'reset', 'measure', 'z', 's', 't', 'rz', 'sdg', 'h', 'x', 'id', 'sx'}
Multi qubit instructions:
  (0, 1): {'cr45m', 'cr45p', 'ecr'}
  (1, 0): {'cr45m', 'cr45p', 'ecr'}
)>
Starting Rabi experiment for qubit 0...
Rabi experiment for qubit 0 done.
Starting Drag experiment for qubit 0...
Drag experiments done for qubit 0 done.
Starting Rabi experiment for qubit 1...
Rabi experiment for qubit 1 done.
Starting Drag experiment for qubit 1...
Drag experiments done for qubit 1 done.
All single qubit calibrations are done
Updated Instruction Schedule Map <InstructionScheduleMap(1Q instructions:
  q0: {'delay', 'tdg', 'reset', 'measure', 'z', 's', 't', 'rz', 'sdg', 'h', 'x', 'id', 'sx'}
  q1: {'delay', 'tdg', 'reset', 'measure', 'z'

In [2]:
gate_q_env_config.target

{'register': [0, 1],
 'gate': Instruction(name='cx', num_qubits=2, num_clbits=0, params=[])}

### Perform HPO

In [3]:
from hyperparameter_optimization import HyperparameterOptimizer

In [4]:
path_agent_config = '/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/template_configurations/agent_config.yaml'
save_results_path = 'hpo_results'

In [5]:
optimizer = HyperparameterOptimizer(gate_q_env_config, path_agent_config, save_results_path, log_progress=True, num_hpo_trials=1)
optimizer.optimize_hyperparameters()

SparsePauliOp(['II', 'XY', 'YX', 'ZZ'],
              coeffs=[ 0.25+0.j,  0.25+0.j, -0.25+0.j, -0.25+0.j])
2023-12-22 16:41:09 INFO num_HPO_trials: 1
2023-12-22 16:41:09 INFO ---------------- STARTING HPO ----------------


[I 2023-12-22 16:41:09,270] A new study created in memory with name: no-name-e699ce32-5949-4df1-bc73-3825f88a543d


SparsePauliOp(['II', 'XX', 'YZ', 'ZY'],
              coeffs=[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j])


  0%|          | 0/41 [00:00<?, ?it/s]

SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ'],
              coeffs=[ 0.25+0.j, -0.25+0.j,  0.25+0.j, -0.25+0.j])


...

mean tensor([-0.0175, -0.0521,  0.0548,  0.0409,  0.0008,  0.0076,  0.0234])
Average return: 0.8965599802734157
DFE Rewards Mean: 0.8965599802734157
DFE Rewards standard dev 0.08901552195752253
Returns Mean: 3.1307304
Returns standard dev 2.5322015
Advantages Mean: 1.4637333
Advantages standard dev 2.5322015
Fidelity History: []
SparsePauliOp(['II', 'IX', 'XI', 'XX'],
              coeffs=[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j])
mean tensor([ 0.0225, -0.0798,  0.0613,  0.0650,  0.0215, -0.0050, -0.0064])
Average return: 0.8044381662642647
DFE Rewards Mean: 0.8044381662642647
DFE Rewards standard dev 0.20415453368402267
Returns Mean: 2.644345
Returns standard dev 2.5238638
Advantages Mean: 0.73219174
Advantages standard dev 2.5238638
Fidelity History: []
SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ'],
              coeffs=[ 0.25+0.j,  0.25+0.j, -0.25+0.j, -0.25+0.j])
mean tensor([-0.0198, -0.0619,  0.0568,  0.0964,  0.0344,  0.0094, -0.0173])
Average return: 0.8517239167530126
DFE Rewards Mean

 95%|█████████▌| 39/41 [00:03<00:00, 10.32it/s]

mean tensor([ 0.0340, -0.1205,  0.0675,  0.0746,  0.0357,  0.0004, -0.0500])
Average return: 0.8592462341786888
DFE Rewards Mean: 0.8592462341786888
DFE Rewards standard dev 0.12058967184679621
Returns Mean: 2.624636
Returns standard dev 2.0748253
Advantages Mean: 0.2185318
Advantages standard dev 2.0748253
Fidelity History: []
SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ'],
              coeffs=[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j])
mean tensor([ 0.0121, -0.0726,  0.0542,  0.0559,  0.0344, -0.0128, -0.0204])
Average return: 0.919771507547959
DFE Rewards Mean: 0.919771507547959
DFE Rewards standard dev 0.08371806696945931
Returns Mean: 3.4585795
Returns standard dev 2.4844203
Advantages Mean: 1.186446
Advantages standard dev 2.48442
Fidelity History: []
SparsePauliOp(['II', 'IZ', 'XI', 'XZ'],
              coeffs=[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j])
mean tensor([ 0.0161, -0.0548,  0.0747,  0.0179,  0.0162, -0.0195, -0.0305])
Average return: 0.9705688684334822
DFE Rewards Mean: 0.970568

100%|██████████| 41/41 [00:04<00:00, 10.01it/s]
[I 2023-12-22 16:41:13,446] Trial 0 finished with value: 0.919771507547959 and parameters: {'N_UPDATES': 41, 'N_EPOCHS': 8, 'MINIBATCH_SIZE': 96, 'BATCHSIZE_MULTIPLIER': 5, 'LR': 0.004328386484390127, 'GAMMA': 0.982912259126364, 'GAE_LAMBDA': 0.9509902702390876, 'ENT_COEF': 0.0005003431067605046, 'V_COEF': 0.37482712101861676, 'GRADIENT_CLIP': 0.3118868436386038, 'CLIP_VALUE_COEF': 0.18832393522435506, 'CLIP_RATIO': 0.11896196318308204}. Best is trial 0 with value: 0.919771507547959.


mean tensor([ 0.0221, -0.0907,  0.0814,  0.0091,  0.0283, -0.0079, -0.0325])
Average return: 0.9840382269285838
DFE Rewards Mean: 0.9840382269285838
DFE Rewards standard dev 0.02022401408049091
Returns Mean: 6.1708503
Returns standard dev 3.892446
Advantages Mean: 3.0529683
Advantages standard dev 3.8924463
Fidelity History: []
SparsePauliOp(['II', 'IX', 'XI', 'XX'],
              coeffs=[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j])
mean tensor([ 0.0313, -0.1003,  0.0890,  0.0042,  0.0279, -0.0144, -0.0613])
Average return: 0.937077008492953
DFE Rewards Mean: 0.937077008492953
DFE Rewards standard dev 0.07634692418079787
Returns Mean: 4.382648
Returns standard dev 3.418599
Advantages Mean: 0.71017724
Advantages standard dev 3.4185987
Fidelity History: []
2023-12-22 16:41:13 INFO ---------------- FINISHED HPO ----------------
2023-12-22 16:41:13 INFO HPO completed in 4.18 seconds.
2023-12-22 16:41:13 INFO Best trial:
2023-12-22 16:41:13 INFO -------------------------
2023-12-22 16:41:13 INF

In [10]:
optimizer.target_gate

{'target_gate': Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]),
 'target_register': [0, 1]}

In [6]:
optimizer.hyperparams

['N_UPDATES',
 'N_EPOCHS',
 'MINIBATCH_SIZE',
 'BATCHSIZE_MULTIPLIER',
 'LR',
 'GAMMA',
 'GAE_LAMBDA',
 'ENT_COEF',
 'V_COEF',
 'GRADIENT_CLIP',
 'CLIP_VALUE_COEF',
 'CLIP_RATIO',
 'BATCHSIZE']

In [7]:
optimizer.num_hpo_trials

1

In [8]:
optimizer.best_hpo_configuration

{'best_avg_return': 0.919771507547959,
 'best_hyperparams': {'N_UPDATES': 41,
  'N_EPOCHS': 8,
  'MINIBATCH_SIZE': 96,
  'BATCHSIZE_MULTIPLIER': 5,
  'LR': 0.004328386484390127,
  'GAMMA': 0.982912259126364,
  'GAE_LAMBDA': 0.9509902702390876,
  'ENT_COEF': 0.0005003431067605046,
  'V_COEF': 0.37482712101861676,
  'GRADIENT_CLIP': 0.3118868436386038,
  'CLIP_VALUE_COEF': 0.18832393522435506,
  'CLIP_RATIO': 0.11896196318308204}}